In [17]:
# Import all library.

import ROOT
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# Read a data file and get all variables.
data = '/home/isabella/Workspace/Adrian2015/Public/WH/output/local/readPaul_1_0_J1Pt45+2BTag+TruthGENWZ+Clean_1_perevent+perjet/lvbb125.root'

treeName = 'perevent'
file = ROOT.TFile(data,"READ")
tree = file.Get(treeName)
#nrEvents = tree.GetEntries()
nrEvents = 4

for (i,entry) in enumerate(tree):
    if nrEvents>0:
        if i>nrEvents:
            break
    
    j1reco_Pt = getattr(entry,'j1_EMJESGSCMu_Pt')
    j1reco_Eta = getattr(entry,'j1_EMJESGSCMu_Eta')
    j1reco_Phi = getattr(entry,'j1_EMJESGSCMu_Phi')
    j1reco_E = getattr(entry,'j1_EMJESGSCMu_E')
    
    j2reco_Pt = getattr(entry,'j2_EMJESGSCMu_Pt')
    j2reco_Eta = getattr(entry,'j2_EMJESGSCMu_Eta')
    j2reco_Phi = getattr(entry,'j2_EMJESGSCMu_Phi')
    j2reco_E = getattr(entry,'j2_EMJESGSCMu_E')
    
    
    
    #FracEM3 = 
    #FracTile0 =
    #TrkWidth =
    #EMF =
    #JVF = 
    #NTrk = 
    #SumPtTrk =
    

In [11]:
# Using Txt file in order to make a correlative.

data = '/home/isabella/Workspace/python/PtRelAnalysis/PtRelAnalysis/Results/TxtFiles/TxtFile_lvbb125.txt'

In [31]:
# Import a Matrix with variables from txt file using numpy

f = open(data,'r')

M = np.loadtxt(data)

B = M.T
AutCor = np.corrcoef(M,rowvar=0)

plt.acorr(M)


ValueError: object too deep for desired array